# 0312
- 회귀 문제에서 사용되는 교차 검증
- 데이터의 편중을 막기 위해!
- KFold, StratifiedKFold 사용

In [1]:
from sklearn.datasets import load_iris

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.metrics import classification_report

import numpy as np
import pandas as pd

## 교차검증(KFold)
- 회귀 문제에서 사용
- train/test로 나누게 되면 과적합이 발생할 확률이 높다(overfitting)
- 이런 부분을 해결하기 위해서 교차검증을 진행
- 교차검증은 데이터의 편중을 막기위해 여러 개의 별도의 세트로 구성된 학습 데이터와 검증 데이터 세트로 학습과 평가를 수행하는 것
- 그 수행결과에 따라 하이퍼파라미터를 튜닝하는 등 모델 최적화를 진행

In [4]:
iris=load_iris()
fold_iris=iris.data
label=iris.target

fold_clf=DecisionTreeClassifier()

### 교차검증 코드

In [11]:
kfold=KFold(n_splits=5)     # 5번 폴드 세트별로 정확도를 볼 것이다.

cv_accuracy=[]     # 정확도
cv_precision=[]    # 정밀도
cv_recall=[]       # 재현율

kfold.split(fold_iris)      # 나누기?

<generator object _BaseKFold.split at 0x0000028819182560>

In [12]:
n_iter=0
for train_idx,test_idx in kfold.split(fold_iris):
    x_train,x_test=fold_iris[train_idx],fold_iris[test_idx]
    y_train,y_test=label[train_idx],label[test_idx]

    # 학습 진행
    fold_clf.fit(x_train,y_train)
    #예측
    fold_pred=fold_clf.predict(x_test)

    # 정확도 측정
    n_iter +=1
    accuracy=np.round(accuracy_score(y_test,fold_pred),5)
    #precision = np.round(precision_score(y_test,fold_pred),5)
    #recall = np.round(recall_score(y_test, fold_pred),5)
    print('\n{} 교차검증 정확도 : {}, 교차검증 precision :{}, 교차검증 recall :{}'.format(n_iter, accuracy, precision, recall))
    cv_accuracy.append(accuracy)
    #cv_precision.append(precision)
    #cv_recall.append(recall)


1 교차검증 정확도 : 1.0, 교차검증 precision :[], 교차검증 recall :[]

2 교차검증 정확도 : 1.0, 교차검증 precision :[], 교차검증 recall :[]

3 교차검증 정확도 : 0.9, 교차검증 precision :[], 교차검증 recall :[]

4 교차검증 정확도 : 0.93333, 교차검증 precision :[], 교차검증 recall :[]

5 교차검증 정확도 : 0.73333, 교차검증 precision :[], 교차검증 recall :[]


- recall, precision 은 타겟값이 이진이 아니면 계산할 때 문제가 생긴다.

In [19]:
iris_df=pd.DataFrame(data=iris.data,columns=iris.feature_names)     # iris 데이터의 컬럼명을 열로 해서 데이터프레임 생성
iris_df['target']=iris.target       # 타겟 컬럼 생성
iris_df
print(iris_df['target'].value_counts())     # 타겟값의 종류별 개수


0    50
1    50
2    50
Name: target, dtype: int64


In [21]:
# kfold의 문제점

kfold_iris=KFold(n_splits=3)
kfold_iris.split(iris_df)

cnt_iter=0
for train_idx,test_idx in kfold_iris.split(iris_df):
    cnt_iter+=1
    label_train=iris_df['target'].iloc[train_idx]
    label_test = iris_df['target'].iloc[test_idx]
    print('교차검증 :{}'.format(cnt_iter))
    print('학습 레이블데이터 분포\n', label_train.value_counts())
    print('검증 레이블데이터 분포\n', label_test.value_counts())

교차검증 :1
학습 레이블데이터 분포
 1    50
2    50
Name: target, dtype: int64
검증 레이블데이터 분포
 0    50
Name: target, dtype: int64
교차검증 :2
학습 레이블데이터 분포
 0    50
2    50
Name: target, dtype: int64
검증 레이블데이터 분포
 1    50
Name: target, dtype: int64
교차검증 :3
학습 레이블데이터 분포
 0    50
1    50
Name: target, dtype: int64
검증 레이블데이터 분포
 2    50
Name: target, dtype: int64


- kfold의 문제점 
    -> 정답비중에 대해서 고르게 나눠지는 게 아니라 랜덤하게 나눠짐, kfold 나눴을 때 0,1 만 포함되거나 0만 포함되거나, 2만 포함되거나 1,2 포함되거나 이런식으로 데이터가 나눠질 수 있다.

## StratifiedKFold
- 분류 문제에서 사용
- target 값에서 데이터가 한 쪽으로 몰리지 않게

In [22]:
from sklearn.model_selection import StratifiedKFold

In [23]:
skf_iris=StratifiedKFold(n_splits=3)

cnt_iter=0
for train_idx,test_idx in skf_iris.split(iris_df,iris_df['target']):
    cnt_iter += 1
    label_train = iris_df['target'].iloc[train_idx]
    label_test = iris_df['target'].iloc[test_idx]
    print('교차검증 :{}'.format(cnt_iter))
    print('학습 레이블데이터 분포\n', label_train.value_counts())
    print('검증 레이블데이터 분포\n', label_test.value_counts())

교차검증 :1
학습 레이블데이터 분포
 2    34
0    33
1    33
Name: target, dtype: int64
검증 레이블데이터 분포
 0    17
1    17
2    16
Name: target, dtype: int64
교차검증 :2
학습 레이블데이터 분포
 1    34
0    33
2    33
Name: target, dtype: int64
검증 레이블데이터 분포
 0    17
2    17
1    16
Name: target, dtype: int64
교차검증 :3
학습 레이블데이터 분포
 0    34
1    33
2    33
Name: target, dtype: int64
검증 레이블데이터 분포
 1    17
2    17
0    16
Name: target, dtype: int64


In [30]:
result_skfold=StratifiedKFold(n_splits=3)
result_clf=DecisionTreeClassifier(random_state=100)
n_iter=0

cv_accuracy=[]
cv_precision=[]
cv_recall=[]

for train_idx, test_idx in result_skfold.split(fold_iris, label):
    X_train, X_test = fold_iris[train_idx], fold_iris[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    
    #학습을 진행
    result_clf.fit(X_train, y_train)
    #예측
    pred = result_clf.predict(X_test)
    
    #정확도 관련해서 측정
    n_iter +=1 
    accuracy = np.round(accuracy_score(y_test, pred),3)
    precision = np.round(precision_score(y_test,pred, average='weighted'),3)
    recall = np.round(recall_score(y_test, pred, average='weighted'),3)
    print('\n{} 교차검증 정확도 : {}, 교차검증 precision :{}, 교차검증 recall :{}'.format(n_iter, accuracy, precision, recall))
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print('\n')
#print('\n 평균 정확도 ', np.mean(accuracy))
print('\n 평균 검증 정확도', np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))


1 교차검증 정확도 : 0.98, 교차검증 precision :0.981, 교차검증 recall :0.98

2 교차검증 정확도 : 0.92, 교차검증 precision :0.92, 교차검증 recall :0.92

3 교차검증 정확도 : 0.96, 교차검증 precision :0.96, 교차검증 recall :0.96



 평균 검증 정확도 0.9533333333333333 0.9536666666666666 0.9533333333333333


- 타겟 값이 이진이라면 문제가 없지만, 그 이상이라면 precision, recall 값 계산에 오류가 생긴다.
- 다르게 계산하기 위해서는 average= 를 이용
    - micro(전체의 평균)
    - macro(평균 중의 평균)
    - weighted (가중 평균)